In [ ]:
import os
import time
from xpander_sdk import XpanderClient, LLMProvider, OpenAISupportedModels
from openai import OpenAI
from typing import List, Dict, Any
import logging
from dotenv import load_dotenv
from datetime import datetime
import logging
import re
import json

# Load environment variables
load_dotenv()
# Setup Logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger(__name__)

# Load environment variables
openai_client = OpenAI(api_key= os.environ.get("OPENAI_API_KEY", ""))
xpander_client = XpanderClient(api_key=os.environ.get("XPANDER_API_KEY", ""))
xpander_agent = xpander_client.agents.get(agent_id=os.environ.get("XPANDER_AGENT_ID", ""))

class SharedMemory:
    def __init__(self):
        self.memory = []

    def add_message(self, content: str, role: str = "assistant", agent_name: str = None):
        self.memory.append({"role": role, "content": content, "agent_name": agent_name})

    def get_memory(self) -> list:
        return self.memory


class PlannerAgent:
    def __init__(self, handler, tools: list, task_message: str, system_message: str, shared_memory: SharedMemory,
                 finish_message: str = "Final Answer"):
        self.handler = handler
        self.tools = tools
        self.task_message = task_message
        self.system_message = system_message
        self.local_memory = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": task_message},
        ]
        self.finish_message = finish_message
        self.is_finished = False
        self.step_number = 1
    def invoke_llm(self, memory, tools=None, model=OpenAISupportedModels.GPT_4_O, max_tokens=16384):
        try:
            response = self.handler.chat.completions.create(
                model=model,
                messages=memory,
                tools=tools,
                tool_choice="none",
                max_tokens=max_tokens,
                temperature=0.0,
                top_p=1
            )
            return response
        except Exception as e:
            raise RuntimeError(f"Error invoking LLM: {e}")
        
    def run_post_processing(self, response: str) -> str:
        """
        Post-process the LLM response to check for completion.
        """
        if re.search(self.finish_message, response):
            self.is_finished = True 
        return response

    def finished(self) -> bool:
        """
        Check if the agent has completed its task.
        """
        return self.is_finished



class ToolSelectorAgent:
    def __init__(self, handler, tools: list, task_message: str, system_message: str, shared_memory: SharedMemory):
        self.handler = handler
        self.tools = tools
        self.task_message = task_message
        self.system_message = system_message
        self.local_memory = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": task_message},
        ]
        self.selected_tools = []

    def invoke_llm(self, memory, tools=None, model=OpenAISupportedModels.GPT_4_O, max_tokens=16384):
        try:                                
            response=self.handler.chat.completions.create(
                model=model,
                messages=memory,
                tools=tools,
                parallel_tool_calls=False,
                tool_choice="required",
                max_tokens=max_tokens,
                temperature=0.0,
                top_p=1
            )
            return response
        except Exception as e:
            raise RuntimeError(f"Error invoking LLM: {e}")

class ParserAgent:
    def __init__(self, handler, tools: list, task_message: str, system_message: str, shared_memory: SharedMemory):
        self.handler = handler
        self.task_message = task_message
        self.system_message = system_message
        self.tools = tools
        self.local_memory = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": task_message},
        ]
    def invoke_llm(self, memory, tools=None, model=OpenAISupportedModels.GPT_4_O, max_tokens=16384):
        try:
            response=self.handler.chat.completions.create(
                model=model,
                messages=memory,
                max_tokens=max_tokens,
                temperature=0.0,
            )
            return response
        except Exception as e:
            raise RuntimeError(f"Error invoking LLM ParserAgent: {e}")

In [2]:
system_prompt_planner = '''As a planner agent your task is to breakdown the main task into sub tasks that fulfill the main task. 
your main task is to build a comprehensive report on the topic provided by the user '{user_query}' '''

tool_selector_system_prompt = '''You are an tool selector agent responsible for selecting the correct tool with the most relevant parameters that will fullfil the task your will received
from the planner agent to execute the plan.'''

parser_system_prompt = '''You are a parser agent that get to get the tool execution response from the relevant tool and generate a human readable report about the required
user request/topic '{user_query}' based on the tool response and the plan in this current step.'''

planner_task_prompt = '''
As a planner research agent your task is to plan the the full workflow for the tool execution agent and the parser agent for making a comprehensive report on the topic provided by the user '{user_query}'
you need to explain each step in the workflow, what is the current required data and what are the expected results.

what expected from you:
 
Produce a high-quality, comprehensive report that addresses the user’s query in-depth, using the latest and most relevant information gathered from multiple sources. The report should be informative, fact-based, and structured cohesively, with clear sections.
Available Tools for Data Gathering:
You have access to the following tools to collect data on the topic:
   - **Tavily**: conduct a search on the topic and get a summary of the relevant information.
   - **Arxiv**: Fetch recent articles within the last year on the topic focusing on their introductory sections.
   - **Perplexity**: Gather additional, complementary data that may not appear in Tavily or Arxiv results.
   - **LinkedIn**: Search for posts from professionals and organizations that provide real-world insights and can enrich the report.
Use each tool to gather data relevant to the user’s query. after all tools are used you must create a PDF report using the Markdown-to-PDF tool the expected output is a link to the PDF report.

These are the strict rules:
1. Always provide your plan in natural language, ensuring it is closely related to the input tools, you must related to the available tools you got.
2. Be specific in the plan and explain what should be the results of this step after parsing the API response. 
3. User's query can't be fulfilled without at least one 'Parser Response'. You can't fulfill task without at least one API calling and response.
4. If the query has not been fulfilled, explain how to fix the last step and continue to output your plan.
5. Return only the next Plan step (i+1) you generated and do not mention all the steps list until now. you'll get the conversation history Plan steps [1,...,i-1] and API responses after parsing, you will use it to generate the next step.
6. If the the API request failed, return how you recommend to handle this error in the next retry of the tool calling.
7. you must return only one step in each call! never return the full step pipeline in one iteration.
8. after you collect all the information you must create the final PDF report using the Markdown-to-PDF tool.
9. You MUST return the Final Answer after using all the tools: Tavily, Arxiv, Perplexity and LinkedIn and Markdown-to-PDF tool !

this is the expected output template:
if the query has not been fulfilled:
Plan step (i+1): [the next step of your plan for how to solve the query].

if all the data is collected and 'Markdown-to-PDF' tool is used and the Parser returned link to the PDF report you MUST return the final answer with the link to the PDF report that got from the Parser.
you MUST return by the following template: Final Answer: [link to the PDF report].

please start with the first step and return only one step in each call!
your research on the topic {user_query} begin now...
'''

tool_selector_task_prompt = '''
as a part of a multi agent system your task is to get as much information as possible on the user topic {user_query} and create a comprehensive PDF report.

Your task:
1. Select the most accurate tool to fulfill the current task provided by the planner agent pay attention to the tools the planner agent provided you. 
2. Generate all required parameters by the schema you got that will fulfill the task.
3. You must return your answer as a tool_call with the function name and relevant arguments.
4. If the planer explain about the error and how to fix it, you should fix the last tool call parameters and return the new tool call.
5. when you creating the final report you must used all the previews information that collected in the parserAgent response steps.
6. if tool call is success you must use each tool only once!
here the required user topic: {user_query}
'''

parser_task_prompt = '''
as part of the multi agent pipeline you are the parser agent and your task is to get the current step plan and the response from the tool that executed for this plan and parse this response to fulfill the plan.
your response should contains all the relevent details for the report without missing information.
your response should contains as much details as you can for creating the final report.
the report must be informative and clear as much as you can and stick with the plan.

These are the strict rules:
1. when you creating the final report you must use all the information you got until now on the user topic {user_query}.
2. you must return the report as human readable text that include all the information.
3. Confirm that the report comprehensively answers the user’s query and provides an in-depth view based on the gathered data.
4. Verify that all sections flow logically, contributing to a unified, fact-based narrative without tool-specific attributions.
5. Ensure that the report includes a structured introduction and conclusion, and that it is cohesive, informative, and thoroughly addresses the main topic.
6. Do NOT add your own words to the report!  you must use only the information you got from the tools.
7. Ensure that the report is formatted correctly and is easy to read and the main title is bold and larger than the other titles.
8. after you use 'Markdown-to-PDF' tool you must tell the planner agent that you finished and return only the link to the PDF report that got from the tool. DO NOT RETURN ANYTHING ELSE!

here the required user topic: {user_query}
'''

In [ ]:
## User Query
user_query = "create a report on AI Agents"

start_time = time.time()
tools = xpander_agent.get_tools()
current_date = datetime.now().date()
shared_memory = SharedMemory()

# Initialize Agents
planner_agent = PlannerAgent(handler=openai_client,tools=tools, task_message=planner_task_prompt.format(user_query=user_query), system_message=system_prompt_planner.format(user_query=user_query),shared_memory=shared_memory)
tool_selector_agent = ToolSelectorAgent(handler=openai_client,tools=tools,task_message=tool_selector_task_prompt.format(user_query=user_query),system_message=tool_selector_system_prompt , shared_memory=shared_memory)
parser_agent = ParserAgent(handler=openai_client, tools=tools, task_message=parser_task_prompt.format(user_query=user_query) ,system_message=parser_system_prompt.format(user_query=user_query), shared_memory=shared_memory)


planner_response = planner_agent.invoke_llm(memory=planner_agent.local_memory,tools=planner_agent.tools)
shared_memory.add_message(planner_response.choices[0].message.content,role="assistant",agent_name="PlannerAgent")
logger.info("Planner Response: %s", planner_response.choices[0].message.content)

tool_selector_response = tool_selector_agent.invoke_llm(memory=tool_selector_agent.local_memory+shared_memory.get_memory(),tools=tools)
tool_calls = xpander_client.extract_tool_calls(llm_response=tool_selector_response.model_dump(), llm_provider=LLMProvider.OPEN_AI)
_ = xpander_agent.run_tools(tool_calls)


In [ ]:
shared_memory = SharedMemory()
tools = xpander_agent.get_tools()


planner_response = planner_agent.invoke_llm(memory=planner_agent.local_memory,tools=tools)
shared_memory.add_message(planner_response.choices[0].message.content,role="assistant",agent_name="PlannerAgent")
logger.info("Planner Response: %s", planner_response.choices[0].message.content)

while not planner_agent.finished():
    try:
        
        tool_selector_response = tool_selector_agent.invoke_llm(memory=tool_selector_agent.local_memory+shared_memory.get_memory(),tools=tools)
        tool_calls = xpander_client.extract_tool_calls(llm_response=tool_selector_response.model_dump(), llm_provider=LLMProvider.OPEN_AI)
        if tool_calls:
            for tool_call in tool_calls:
                logger.info("Tool calls: %s", tool_call.name)
                tool_response = xpander_agent.run_tool(tool_call)
                logger.info("Tool response: %s", tool_response.result)
                
                selected_tool_params = json.dumps(tool_selector_response.model_dump()['choices'][0]['message']['tool_calls'])
                selected_tool_data = {
                    "selected_tool": tool_call.name,
                    "tool_call_id": tool_call.tool_call_id,
                    "params": selected_tool_params,
                    "response": tool_response.result
                }
                logger.info("Tool selector response: %s", json.dumps(selected_tool_data))
                
                parser_message = (
                    f"Current Task: {planner_response.choices[0].message.content}\n"
                    f"The selected Tool: {tool_call.name}\n"
                    f"The params Tool: {json.dumps(selected_tool_data)}\n"
                )

                parser_response = parser_agent.invoke_llm(memory=parser_agent.local_memory+shared_memory.get_memory()+[{"role": "user", "content": parser_message}],tools=tools)
                shared_memory.add_message(parser_response.choices[0].message.content,role="assistant",agent_name="ParserAgent")
                logger.info("Parser response: %s", parser_response.choices[0].message.content)


    except Exception as e:
        logger.error(f"Pipeline error: {e}")
        break
    
    tools = xpander_agent.get_tools()
    planner_response = planner_agent.invoke_llm(memory=planner_agent.local_memory+shared_memory.get_memory(),tools=tools)
    shared_memory.add_message(planner_response.choices[0].message.content,role="assistant",agent_name="PlannerAgent")
    logger.info("Planner response: %s", planner_response.choices[0].message.content)
    
    planner_agent.run_post_processing(planner_response.choices[0].message.content)
    

logger.info("Pipeline execution complete!")
logger.info("Shared Memory:")
logger.info(shared_memory.get_memory())
